In [2]:
import pandas as pd
import numpy as np
import boto3
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
s3 = boto3.resource('s3')
mrds_csv = pd.read_csv('../data/mrds.csv')

/tmp/ipykernel_2121/2462994313.py:2: DtypeWarning: Columns (3,17,25,26,28,34,35,37,39,41,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  mrds_csv = pd.read_csv('../data/mrds.csv')


In [4]:
df =(
    mrds_csv[['latitude','longitude','state','com_type','commod1','commod2','commod3']]
    .query("state == 'Colorado'")
)

In [5]:
def combine_cols(row):
    arr =[]
    if not pd.isna(row['commod1']):
        arr.extend(str(row['commod1']).split('-'))
    if not pd.isna(row['commod2']):
        arr.extend(str(row['commod2']).split('-'))
    if not pd.isna(row['commod3']):
        arr.extend(str(row['commod3']).split('-'))
    return arr

In [6]:
commod_X = (
    df[['latitude','longitude', 'com_type']]
    .assign(cat_codes = pd.Categorical(df['com_type']).codes)
    .drop(['com_type'], axis=1)
)

In [7]:
commod_array = df.apply(combine_cols,axis=1).values.tolist()
commod_array = [[x.strip() for x in l] for l in commod_array]
commod_mlb = MultiLabelBinarizer()
commod_mlb_arr = commod_mlb.fit_transform(commod_array)
commod_cols = commod_mlb.classes_
commod_label_df = pd.DataFrame(commod_mlb_arr, columns=commod_cols)

In [ ]:
commod_X.to_csv('../data/commod_X.csv', index=False)
s3.meta.client.upload_file('../data/commod_X.csv', 'nb-scripts', 'csv_data/commod_X.csv')
commod_label_df.to_csv('../data/commod_labels.csv', index=False)
s3.meta.client.upload_file('../data/commod_labels.csv', 'nb-scripts', 'csv_data/commod_labels.csv')